In [1]:
from ase import Atoms
from ase.build import add_adsorbate, fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 12:30:03       11.689927       1.0797
BFGSLineSearch:    1[  2] 12:30:03       11.670814       0.4090
BFGSLineSearch:    2[  4] 12:30:03       11.625880       0.0409
Adsorption energy: 0.32351942231763786


# Getting started with GPAW

Author : Rakshit Kr. Singh : [Website](https://blog.greatrsingh.in/) : [LinkedIn](https://www.linkedin.com/in/rakshit-singh-ai/) : [GitHub](https://github.com/GreatRSingh)

In [55]:
# creates: h2.emt.traj
from ase import Atoms
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton

system = Atoms('H2', positions=[[0.0, 0.0, 0.0],
                                [0.0, 0.0, 0.5]])
calc = EMT()

system.calc = calc

opt = QuasiNewton(system, trajectory='h2.emt.traj')

opt.run()

print("\nFinal Positions:\n", system.positions)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:27:58        8.805192      78.3340
BFGSLineSearch:    1[  1] 15:27:58        1.546114       6.6223
BFGSLineSearch:    2[  4] 15:27:58        1.079132       1.1951
BFGSLineSearch:    3[  6] 15:27:58        1.070555       0.0491

Final Positions:
 [[ 0.          0.         -0.13979722]
 [ 0.          0.          0.63979722]]


In [62]:
from ase.build import molecule
from gpaw import GPAW, PW

a = 8.0
h = 0.2

energies = {}
with open(f'results-{h:.2f}.txt', 'w') as resultfile:

    for name in ['H2O', 'H', 'O']:
        system = molecule(name)
        system.set_cell((a, a, a))
        system.center()

        if name in ['H', 'O']:
            hund = True
        else:
            hund = False
        calc = GPAW(mode=PW(350), h=h, hund=hund,
                    txt=f'gpaw-{name}-{h:.2f}.txt')

        system.calc = calc

        energy = system.get_potential_energy()
        energies[name] = energy
        print(name, energy, file=resultfile)

    e_atomization = energies['H2O'] - 2 * energies['H'] - energies['O']
    print(e_atomization, file=resultfile)

In [ ]:
H2O -15.253640595828836
H -0.8814511312195988
O -1.830234131376093
-11.660504202013545


In [ ]:
H2O -10.94488101486136
H -0.8650121187990646
O 2.1329098108515128
-11.347766588114744


In [64]:
"""Bulk Al(fcc) test"""
from ase import Atoms
from ase.visualize import view
from gpaw import GPAW, PW

name = 'Al-fcc'
a = 4.05  # fcc lattice parameter
b = a / 2

bulk = Atoms('Al',
             cell=[[0, b, b],
                   [b, 0, b],
                   [b, b, 0]],
             pbc=True)

view(bulk)

k = 4
calc = GPAW(mode=PW(300),       # cutoff
            kpts=(k, k, k),     # k-points
            txt=name + '.txt')  # output file

bulk.calc = calc

energy = bulk.get_potential_energy()
calc.write(name + '.gpw')
print('Energy:', energy, 'eV')

Energy: -4.122151616077782 eV
